In [1]:
import tensorflow as tf

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

import pathlib

from tensorflow.keras.datasets import mnist
import PIL.Image

from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from IPython.display import clear_output
import pydot
from pydot import print_function

Applications:
    
1. Image Classifications
2. Transfer Learning
3. Object Detection

#### Conv1D, Conv2D and Conv3D

$W2=(W1 - F + 2P)/S$+1

$H2=(W1 - F + 2P)/S$+1

OutputShape = (W2, H2, K)

k=> Number of filters

###### Conv1D

    This layer creates a convolution kernel that is convolved with the layer input over a single spatial 
    (or temporal) dimension to produce a tensor of outputs.
    
    Input shape: 3 dimensional (rows, x, y)

In [3]:
input_shape = (3, 3, 3)
x = tf.random.normal(input_shape)
print(x)
y = tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu',input_shape=input_shape)(x)
y.get_shape()

tf.Tensor(
[[[-0.37391165  0.04562892 -1.6824136 ]
  [ 0.44360694 -0.40767294  1.0939786 ]
  [-0.21728949  0.27327123  0.05539514]]

 [[-1.3398204  -0.38405377 -0.33129257]
  [-1.1612829  -0.42730314  0.6373372 ]
  [ 0.7395205   0.08809259 -0.4956954 ]]

 [[-1.0909419   0.42566532  0.01288452]
  [-0.9878779  -0.13403244 -0.9982787 ]
  [ 0.227729   -0.75835276  0.76357263]]], shape=(3, 3, 3), dtype=float32)


TensorShape([3, 1, 32])

###### Conv2D

    This layer creates a convolution kernel that is convolved with the layer input over a two spatial 
    (or temporal) dimension to produce a tensor of outputs.
    
    Input shape: 4 dimensional (rows, x, y, z)

In [ ]:
input_shape = (1, 3, 3, 3)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',input_shape=input_shape)(x)
y.get_shape()

###### Conv3D

    This layer creates a convolution kernel that is convolved with the layer input over a 3 spatial 
    (or temporal) dimension to produce a tensor of outputs.    

In [ ]:
input_shape =(1, 28, 28, 28, 1)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv3D(filters=2, kernel_size=3, activation='relu', input_shape=input_shape)(x)
print(y.shape)

#### Data Augmentation

    Data augmentation is a common technique to improve results and avoid overfitting.
    
    Data augmentation takes the approach of generating more training data from existing training samples by
    augmenting the samples using random transformations that yield believable-looking images. The goal is the model 
    will never see the exact same picture twice during training.
    
In tensorflow data augmentation can be implemented in two ways. 

    Using Keras Preprocessing Layers. 
    tf.image.

In [ ]:
image_path = tf.keras.utils.get_file("cat.jpg", "https://storage.googleapis.com/download.tensorflow.org/example_images/320px-Felis_catus-cat_on_snow.jpg")
PIL.Image.open(image_path)

In [ ]:
image_string=tf.io.read_file(image_path)
image=tf.image.decode_jpeg(image_string, channels=3)

In [ ]:
def visualize(original, augmented):
    fig = plt.figure()
    plt.subplot(1,2,1)
    plt.title('Original image')
    plt.imshow(original)

    plt.subplot(1,2,2)
    plt.title('Augmented image')
    plt.imshow(augmented)

In [ ]:
flipped = tf.image.flip_left_right(image)
visualize(image, flipped)

In [ ]:
grayscaled = tf.image.rgb_to_grayscale(image)
visualize(image, tf.squeeze(grayscaled))
plt.colorbar()

In [ ]:
saturated = tf.image.adjust_saturation(image, 3)
visualize(image, saturated)

In [ ]:
bright = tf.image.adjust_brightness(image, 0.4)
visualize(image, bright)

In [ ]:
rotated = tf.image.rot90(image)
visualize(image, rotated)

In [ ]:
cropped = tf.image.central_crop(image, central_fraction=0.6)
visualize(image,cropped)

#### Image processing

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)

image_count = len(list(data_dir.glob('*/*.jpg')))

print("Images Count: ", image_count)

CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

In [ ]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    label =  parts[-2] == CLASS_NAMES
    return label

def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

for f in list_ds.take(2):
      print(f.numpy())

labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

print('After transformations: ')
for image, label in labeled_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

#### Prepare Data For Training

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10, 10))
    for i in range(image_batch.shape[0]):
        plt.subplot(5, 5, i+1)
        plt.grid(False)
        plt.imshow(image_batch[i], cmap=plt.cm.binary)
        # The CIFAR labels happen to be arrays, 
        # which is why you need the extra index
        plt.xlabel(CLASS_NAMES[label_batch[i]])
        plt.show()

In [ ]:
train_imgs = []
labels = []

for element in labeled_ds:
    train_imgs.append(element[0].numpy())
    labels.append(np.where(element[1].numpy() == True)[0].item())
    
train_imgs = np.asarray(train_imgs)
train_labels = np.asarray(labels)

print("Features: ", train_imgs.shape)
print("Labels: ", train_labels.shape)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(5))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_imgs, train_labels, epochs=10)

In [ ]:
img = train_imgs[:2]
label = CLASS_NAMES[train_labels[:2]]
print(CLASS_NAMES)
print("Target Label: ", label)
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(img)
print(predictions)